In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import os

In [1]:
import tensorflow as tf
print(tf.__version__)

2.16.1


# Initialization

In [9]:
L_min, L_max = 30e-6, 2000e-6
C_min, C_max = 20e-6, 1000e-6
fsw_min, fsw_max = 20e3, 200e3
t_dt_min, t_dt_max = 175.2e-9, 262.8e-9
d_cycle_min, d_cycle_max = 0.2, 0.35

# Array of min and max values for features
min_values = np.array([L_min, C_min, fsw_min, t_dt_min, d_cycle_min])
max_values = np.array([L_max, C_max, fsw_max, t_dt_max, d_cycle_max])


# Path to the model directory
model_dir = 'D:/Krishna/ai-power-converter/new/ml/model/use_this/'

# List of model filenames
model_filenames = [
    'current_delta_current_1.h5',
    'P_CAP_ESR_1.h5',
    'P_COSS_1.h5',
    'P_D.h5',
    'P_G_1.h5',
    'P_L_DCR_1.h5',
    'P_ON_H_P_ON_L.h5',
    'voltage.h5'
]


# Scaling

In [5]:
def custom_scaling(X):
    return np.array([(X[:, 0] - L_min) / (L_max - L_min), 
                     (X[:, 1] - C_min) / (C_max - C_min),
                     (X[:, 2] - fsw_min) / (fsw_max - fsw_min),
                     (X[:, 3] - t_dt_min) / (t_dt_max - t_dt_min),
                     (X[:, 4] - d_cycle_min) / (d_cycle_max - d_cycle_min)]).T

def inverse_custom_scaling(X_scaled):
    X_inverse = np.empty_like(X_scaled)
    X_inverse[:, 0] = X_scaled[:, 0] * (L_max - L_min) + L_min
    X_inverse[:, 1] = X_scaled[:, 1] * (C_max - C_min) + C_min
    X_inverse[:, 2] = X_scaled[:, 2] * (fsw_max - fsw_min) + fsw_min
    X_inverse[:, 3] = X_scaled[:, 3] * (t_dt_max - t_dt_min) + t_dt_min
    X_inverse[:, 4] = X_scaled[:, 4] * (d_cycle_max - d_cycle_min) + d_cycle_min
    return X_inverse

def custom_scaling_1(X):
    # Clip X to ensure values are within the provided min and max values
    X_clipped = np.clip(X, min_values, max_values)
    return (X_clipped - min_values) / (max_values - min_values)

# Predict

In [12]:

# Load the models
models = {}
for filename in model_filenames:
    model_path = os.path.join(model_dir, filename)
    models[filename] = tf.keras.models.load_model(model_path)

# Example input data
input_data = np.array([
    [50e-6, 50e-6, 50e3, 200e-9, 0.25],  # Example data point 1
    [100e-6, 100e-6, 100e3, 220e-9, 0.3]  # Example data point 2
])

# Scale the input data
scaled_input_data = custom_scaling_1(input_data)

# Make predictions using the models
predictions = {}
for filename, model in models.items():
    predictions[filename] = model.predict(scaled_input_data)

# Print the predictions with descriptive names
descriptive_names = {
    'current_delta_current_1.h5': 'Current and Delta Current',
    'P_CAP_ESR_1.h5': 'Capacitor Loss (P_cap_esr)',
    'P_COSS_1.h5': 'Output Capacitance Loss (P_coss)',
    'P_D.h5': 'Dead Time Loss (P_d)',
    'P_G_1.h5': 'Gate Charge Loss (P_g)',
    'P_L_DCR_1.h5': 'Inductor Conduction Loss (P_L_dcr)',
    'P_ON_H_P_ON_L.h5': 'Conduction Loss (P_on_h, P_on_l)',
    'voltage.h5': 'Voltage'
}

for filename, prediction in predictions.items():
    print(f"Predictions from model {descriptive_names[filename]}:")
    print(prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predictions from model Current and Delta Current:
[[3.0896125 3.401791 ]
 [3.5731103 0.7892138]]
Predictions from model Capacitor Loss (P_cap_esr):
[[0.17061475]
 [0.35239658]]
Predictions from model Output Capacitance Loss (P_coss):
[[0.03557235]
 [0.06892209]]
Predictions from model Dead Time Loss (P_d):
[[0.02238342]
 [0.04906586]]
Predictions from model Gate Charge Loss (P_g):
[[0.17953405]
 [0.34830704]]
Predictions from model Inductor Conduction Loss (P_L_dcr):
[[26.419132]
 [34.71471 ]]
Predictions from model Conduction Loss (P_on_h, P_on_l):
[[0.00365901 0.05053134]
 [0.00552162 0.06611356]]
Predictions from model Voltage:
[[3.0978343]
 [3.57976  ]]
